# Automated Feature Engineering

Automated feature engineering is a process of generating thousands of features from multiple tables. 

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import featuretools as ft 

%matplotlib inline

plt.style.use('fivethirtyeight')

In [10]:
clients = pd.read_csv("clients.csv")
loans = pd.read_csv("loans.csv")
payments = pd.read_csv("payments.csv")

### Defining the Entities (Tables)

In [19]:
es = ft.EntitySet(id = 'clients')

# Create an entity from client dataframe
es = es.entity_from_dataframe(entity_id = "clients", 
                         dataframe = clients, 
                         index = "client_id", 
                         time_index = "joined")

# Create an entity from loans dataframe 
es = es.entity_from_dataframe(entity_id = "loans", 
                         dataframe = loans, 
                        variable_types = {'repaid': ft.variable_types.Categorical},
                         index = "loan_id", 
                         time_index = "loan_start")

es = es.entity_from_dataframe(entity_id = "payments", 
                         dataframe = payments, 
                         variable_types = {'missed': ft.variable_types.Categorical},
                         index = "payment_id", 
                         time_index = "payment_date")



2019-01-04 14:40:49,401 featuretools.entityset - WARNING    index payment_id not found in dataframe, creating new integer column


### Defining the Relationships

In [23]:
## Creating the relationship between clients and previous loans 
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                   es['loans']['client_id'])

## Add relationship to entity set
es = es.add_relationship(r_client_previous)

## Relationship between previous loans and previous payments 
r_payments = ft.Relationship(es['loans']['loan_id'],
                            es['payments']['loan_id'])

es = es.add_relationship(r_payments)

2019-01-04 14:44:20,475 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: loans.client_id -> clients.client_id>


### Feature Primitives

In [27]:
primitives = ft.list_primitives()
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,count,aggregation,Counts the number of non null values.
1,std,aggregation,Finds the standard deviation of a numeric feat...
2,all,aggregation,Test if all values are 'True'.
3,last,aggregation,Returns the last value.
4,n_most_common,aggregation,Finds the N most common elements in a categori...
5,min,aggregation,Finds the minimum non-null value of a numeric ...
6,trend,aggregation,Calculates the slope of the linear trend of va...
7,max,aggregation,Finds the maximum non-null value of a numeric ...
8,skew,aggregation,Computes the skewness of a data set.
9,median,aggregation,Finds the median value of any feature with wel...


In [28]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
19,and,transform,"For two boolean values, determine if both valu..."
20,cum_sum,transform,Calculates the sum of previous values of an in...
21,numwords,transform,Returns the words in a given string by countin...
22,subtract,transform,Creates a transform feature that subtracts two...
23,divide,transform,Creates a transform feature that divides two f...
24,cum_min,transform,Calculates the min of previous values of an in...
25,is_null,transform,"For each value of base feature, return 'True' ..."
26,month,transform,Transform a Datetime feature into the month.
27,multiply,transform,Creates a transform feature that multplies two...
28,second,transform,Transform a Datetime feature into the second.


In [29]:
# Create new features using primitives 
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients',
                                agg_primitives = ['mean','max','percent_true','last'],
                                trans_primitives = ['years','month','subtract','divide'])

In [31]:
pd.DataFrame(features)

,income,credit_score,MEAN(loans.loan_amount),LAST(payments.payment_amount),LAST(loans.rate),LAST(loans.loan_amount),MEAN(payments.payment_amount),MAX(loans.loan_amount),MAX(payments.payment_amount),MONTH(joined),...,MEAN(payments.payment_amount) / income - credit_score,MAX(payments.payment_amount) - income,credit_score / MEAN(payments.payment_amount),MEAN(loans.rate) - LAST(payments.payment_amount),MAX(loans.loan_amount) / MEAN(loans.loan_amount),MAX(payments.payment_amount) - MEAN(payments.payment_amount),income - credit_score / MAX(payments.payment_amount),income - credit_score / MEAN(payments.payment_amount),MEAN(loans.MEAN(payments.payment_amount)),MAX(loans.rate) - MEAN(loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,7963.950000,241,7.40,2203,1178.552795,13913,2704,10,...,0.005591,-208718,0.526917,-237.523000,1.746997,1525.447205,77.958950,178.864282,1175.760996,5.963000
26326,227920,633,7270.062500,932,1.45,5275,1166.736842,13464,2658,5,...,0.005133,-225262,0.542539,-929.482500,1.851979,1491.263158,85.510534,194.805711,1161.691608,4.212500
26695,174532,680,7824.722222,2084,0.90,13918,1207.433824,14865,2932,8,...,0.006945,-171600,0.563178,-2081.533889,1.899748,1724.566176,59.294679,143.984703,1221.200615,4.043889
26945,214516,806,7125.933333,1597,2.86,9249,1109.473214,14593,2768,11,...,0.005191,-211748,0.726471,-1594.144667,2.047872,1658.526786,77.207370,192.622947,1115.150112,2.794667
29841,38354,523,9813.000000,800,5.09,7223,1439.433333,14837,2898,8,...,0.038049,-35456,0.363337,-796.555000,1.511974,1458.566667,13.054175,26.281870,1455.233737,3.315000
32726,235705,730,6633.263158,1013,2.83,5325,944.298013,14802,2695,5,...,0.004019,-233010,0.773061,-1009.941053,2.231481,1750.701987,87.189239,248.835639,1014.163680,6.041053
32885,58955,642,9920.400000,1909,9.11,11886,1396.495652,14162,2422,5,...,0.023948,-56533,0.459722,-1906.564000,1.427563,1025.504348,24.076383,41.756664,1438.236984,6.674000
32961,230341,714,7882.235294,223,1.46,1693,1039.053435,14784,2791,4,...,0.004525,-227550,0.687164,-219.069412,1.875610,1751.946565,82.274095,220.996334,1155.584111,5.209412
35089,131176,771,6939.200000,106,7.63,773,1094.748428,13194,2563,3,...,0.008395,-128613,0.704271,-102.486500,1.901372,1468.251572,50.879828,119.118691,1051.937989,4.116500


### Deep Feature Synthesis

In [32]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [33]:
feature_names

[<Feature: income>,
 <Feature: credit_score>,
 <Feature: MEAN(loans.loan_amount)>,
 <Feature: NUM_UNIQUE(loans.repaid)>,
 <Feature: NUM_UNIQUE(loans.loan_type)>,
 <Feature: COUNT(loans)>,
 <Feature: STD(loans.rate)>,
 <Feature: MODE(payments.missed)>,
 <Feature: MEAN(payments.payment_amount)>,
 <Feature: MAX(loans.loan_amount)>,
 <Feature: MAX(payments.payment_amount)>,
 <Feature: COUNT(payments)>,
 <Feature: MONTH(joined)>,
 <Feature: STD(loans.loan_amount)>,
 <Feature: SKEW(loans.rate)>,
 <Feature: SKEW(payments.payment_amount)>,
 <Feature: STD(payments.payment_amount)>,
 <Feature: NUM_UNIQUE(payments.missed)>,
 <Feature: MIN(payments.payment_amount)>,
 <Feature: MODE(loans.repaid)>,
 <Feature: MAX(loans.rate)>,
 <Feature: MIN(loans.rate)>,
 <Feature: WEEKDAY(joined)>,
 <Feature: SUM(loans.loan_amount)>,
 <Feature: YEAR(joined)>,
 <Feature: SUM(payments.payment_amount)>,
 <Feature: MEAN(loans.rate)>,
 <Feature: MIN(loans.loan_amount)>,
 <Feature: MODE(loans.loan_type)>,
 <Feature: SK